In [1]:
import dask
import dask.dataframe as dd
from dask.distributed import Client
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import category_encoders as ce
from numba import njit
from sklearn.pipeline import make_pipeline

ModuleNotFoundError: No module named 'category_encoders'

In [ ]:
client = Client()

In [ ]:
%cd ds1-predictive-modeling-challenge

In [ ]:
x = dd.read_csv('train_features.csv')
y = dd.read_csv('train_labels.csv')
y = y.drop(["id"],axis=1)
y = y.mask(y == "functional needs repair", "non functional")

In [ ]:
len(x), len(y)

In [ ]:
x.columns.values

In [ ]:
x_train, x_test = x.random_split([0.75,0.25],random_state =42)
y_train, y_test = y.random_split([0.75,0.25],random_state =42)

In [ ]:
len(x_train),len(y_train)

In [ ]:
y_train.shape

In [ ]:
%%time
Kn = KNeighborsClassifier(n_neighbors=25,weights='distance')
Kn.fit(x_train[["latitude","longitude"]],y_train)
Kn.predict(x_train[["latitude","longitude"]])
Kn.score(x_test[["latitude","longitude"]],y_test.compute())

In [ ]:
%%time
Kn = KNeighborsClassifier(n_neighbors=25,weights='distance', n_jobs=-1)
Kn.fit(x_train[["latitude","longitude"]],y_train)
Kn.predict(x_train[["latitude","longitude"]])
Kn.score(x_test[["latitude","longitude"]],y_test.compute())

In [ ]:
Kn.score(x_test[["latitude","longitude"]],y_test.compute())

In [ ]:
z = x[["funder", "installer", "permit", "scheme_management","construction_year","payment","latitude","longitude",]]
z = z.mask(z == 0, np.nan).mask(z == "unknown",np.nan)
z = z.isna()
z["poorly_documented"] = False
for i in z.columns.values:
    z["poorly_documented"] = (z["poorly_documented"] | z[i])
x["poorly_documented"] = z["poorly_documented"]

In [ ]:
x["construction_year"] = x["construction_year"].mask(x == 0,2000)

In [ ]:
x.columns

In [ ]:
x_train, x_test = x.random_split([0.75,0.25],random_state =42)
y_train, y_test = y.random_split([0.75,0.25],random_state =42)

In [ ]:
c = ["basin","region","region_code","district_code","scheme_management","extraction_type","management",\
         "extraction_type_group","quantity_group", "payment", "waterpoint_type","water_quality","quality_group","public_meeting"\
        ,"lga","poorly_documented","source","source_class","extraction_type_class","management_group",\
          'gps_height',"population","construction_year"]
cols1 = ["basin","region","region_code","district_code","scheme_management","extraction_type","management",\
         "extraction_type_group","quantity_group", "payment", "waterpoint_type","water_quality","quality_group",\
         "public_meeting","lga","poorly_documented","source","source_class","extraction_type_class", \
         "top_installer","management_group", "top_funder"]
class_weights = {'functional':1,'non functional':1.2,'functional needs repair':1}


In [ ]:

pipeline = make_pipeline(
ce.OneHotEncoder(cols = c,use_cat_names=True),
LogisticRegression(solver='lbfgs',class_weight=class_weights)
)

In [ ]:
%%time
from sklearn.externals import joblib   
with joblib.parallel_backend('dask'):
    pipeline.fit(x_train[c],y_train)
    y_pred = pipeline.predict(x_test[c])
    accuracy_score(y_test,y_pred)

In [ ]:
x_train[c]

In [ ]:
y_train["status_group"].value_counts().compute()

In [ ]:
c = ["latitude","longitude","construction_year"]
from dask_ml.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train[c].values, y_train["status_group"].values)

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000,
                           n_features=500,
                           n_classes=2,
                           n_redundant=250,
                           random_state=42)

from sklearn import linear_model, decomposition
from sklearn.pipeline import Pipeline

logistic = linear_model.LogisticRegression()
pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca),
                       ('logistic', logistic)])


#Parameters of pipelines can be set using ‘__’ separated parameter names:
grid = dict(pca__n_components=[50, 100, 150, 250],
            logistic__C=[1e-4, 1.0, 10, 1e4],
            logistic__penalty=['l1', 'l2'])

# from sklearn.grid_search import GridSearchCV
from dklearn.grid_search import GridSearchCV

estimator = GridSearchCV(pipe, grid)

estimator.fit(X, y)
